In [1]:
from amazon.backend.database import *
from amazon.backend import socketUtils, request, query
from amazon.backend.WorldMessage import WorldMessage
# from amazon.backend.UPSMessage import UPSMessage
from amazon.backend import amazon_ups_pb2 as UPS
from amazon.backend import world_amazon_pb2 as WORLD
from PIL import Image
import io
import copy

import struct

In [2]:
package_id = 0
def buy_product(user_id, product_id, amount, address):
    sock = socketUtils.socket_connect("vcm-30469.vm.duke.edu",29081)
    # modify databse，generate packageid,
    global package_id
    engine = getEngine()
    Session = sessionmaker(bind=engine)
    session = Session()

    # need lock
    # package_id =session.execute(select(func.max(Order.id))).scalar()+1
    package_id += 1
    # 
    neworder = Order(buyer = user_id, product_id = product_id, amount = amount, status = 'packing', package = package_id)
    session.add(neworder)
    session.commit()
    product = session.query(Products).filter(Products.id == product_id).first()

    value = struct.pack('!I', package_id)
    sock.sendall(value)
    value = struct.pack('!I', address[0])
    sock.sendall(value)
    value = struct.pack('!I', address[1])
    sock.sendall(value)
    
    d = sock.recv(4)
    length = struct.unpack('!I', d)[0]
    message = sock.recv(length).decode()
    print(message)
    if(message != 'Success'):
        return message
    

    print(user_id,product_id,amount,address) 
    pass

In [19]:
# buy_product(1, 0, 1, (1,2))
# buy_product(2, 1, 9, (1,4))
buy_product(1, 0, 1, (1,5))
buy_product(2, 0, 2, (1,6))
buy_product(1, 1, 3, (1,7))
buy_product(2, 1, 5, (1,8))
# buy_product(1, 1, 3, (1,9))
# buy_product(2, 1, 4, (1,10))

connect to vcm-30469.vm.duke.edu
Success
1 0 1 (1, 5)
connect to vcm-30469.vm.duke.edu
Success
2 0 2 (1, 6)
connect to vcm-30469.vm.duke.edu
Success
1 1 3 (1, 7)
connect to vcm-30469.vm.duke.edu
Success
2 1 5 (1, 8)
